
## Find data for soggetti in lista aiuti MISE

In [ ]:
## Runs with "daf" enviroment in repo

%config Completer.use_jedi = False #fix TAB slowness with big frames

import numpy as np
import pandas as pd
import seaborn as sns
import glob
import gc #garbage collector
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)

def load_and_stack_data(prefix):
    fileList = glob.glob('./../pac_opencoesione/%s*.csv' % prefix)
    
    if prefix == 'progetti':
        dtypesDict = pd.read_csv(
            './openCoesione_dtypes.csv', index_col='Variabile', squeeze=True).to_dict()
        dateCols = [name for name in dtypesDict.keys() if dtypesDict[name] == 'datetime64']
        for name in dateCols: del dtypesDict[name]
        frames = list(map(lambda f: pd.read_csv(
            f, sep=';', dtype=dtypesDict, index_col=0, na_values=[' ', '  ']), fileList))
        data = pd.concat(frames, axis=0)
        data[dateCols] = data[dateCols].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
        
        # drop duplicates including index when checking
        indexColName = data.index.name
        data.reset_index(inplace=True)
        data.drop_duplicates(inplace=True)
        data.set_index(indexColName, inplace=True)
        
        # clean remaining duplicate indexes by picking the row that has fewer N/A
        bDuplicate = data.index.duplicated(keep=False)
        if any(bDuplicate):
            print('%i duplicate indexes found' % sum(bDuplicate))
            duplRows = data[bDuplicate].copy()
            duplRows['NumberNA'] = duplRows.isna().sum(axis=1)
            duplRows.sort_values('NumberNA', inplace=True)
            # recomibine into final frame
            data = pd.concat([data[~bDuplicate], duplRows[duplRows.index.duplicated(keep='first')]])
        
    else:
        # infer types for other datasets (temporary)
        frames = list(map(lambda f: pd.read_csv(f, sep=';', index_col=False), fileList))
        data = pd.concat(frames, axis=0)
        
        # drop duplicates excluding index
        data.drop_duplicates(inplace=True)
        
    print('Imported %s' % prefix)
    return data



In [ ]:
listaMise =  pd.read_csv('./../MISE Aiuti/data/D_general/ListIvaCodFisc.csv', sep=';', index_col=False)

# load soggetti from OpenCoesione
soggetti = load_and_stack_data('soggetti')

In [ ]:
print('Numero Match: %i' % len(listaMise.values[listaMise.isin(soggetti.OC_CODICE_FISCALE_SOGG.values)]))

datiCompletiMatch = soggetti[soggetti.OC_CODICE_FISCALE_SOGG.isin(listaMise.CODICE_FISCALE_BENEFICIARIO)]
print(datiCompletiMatch.head(20))


## Soggetti con piu' progetti

In [ ]:
print(datiCompletiMatch.groupby('OC_CODICE_FISCALE_SOGG')['COD_LOCALE_PROGETTO'].nunique().sort_values(ascending=False))
